In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from pprint import pprint
import requests
import re
from random import *

## beautiful soup을 이용하여 이미지의 이름만 따기

In [2]:
# requests를 통해 링크 들어가기
html=requests.get("https://www.ewg.org/skindeep/product/671517/Aromatica_95%25_Organic_Aloe_Vera_Gel/")

In [2]:
# beautifulsoup으로 html을 파싱한다.
soup=BeautifulSoup(html.text,'html.parser')

In [4]:
#requests문 닫기
html.close()

In [24]:
#beautifulsoup을 통해 받은 것들 중에 필요한 이미지 파일 이름을 얻기 위해 그에 맞는 것을 찾아서 집어 넣어 준다.
#ewg 이미지는 div class에서 id로 시작하는 곳에 위치하여 그곳의 경로를 지정해서 보았다.
data1_list=soup.findAll('div',{'id':"prod_cntr_score"})

In [25]:
data1_list

[<div id="prod_cntr_score"><div style="text-align: center;"><a class="preview" href="/skindeep/common/read_score.php" rel="#overlay"><img src="https://static.ewg.org/skindeep/img/draw_score/score_image1__1_.png" style="border: 0"/></a></div><div id="data_avail_box"><div id="score_style_small" style="color:#fff;">Data: <span style="color:#e66e25;">Limited</span></div></div></div>,
 <div id="prod_cntr_score"><div style="text-align: center;"><a class="preview" href="/skindeep/common/read_score.php" rel="#overlay"><img src="https://static.ewg.org/skindeep/img/draw_score/score_image1__1_.png" style="border: 0"/></a></div><div id="data_avail_box"><div id="score_style_small" style="color:#fff;">Data: <span style="color:#e66e25;">Limited</span></div></div></div>,
 <div id="prod_cntr_score"><div style="text-align: center;"><a class="preview" href="/skindeep/common/read_score.php" rel="#overlay"><img src="https://static.ewg.org/skindeep/img/draw_score/score_image1__1_.png" style="border: 0"/></a

In [176]:
#길기 때문에 짧게 하기 위해 다시 한 번 findAll을 사용하여 추출한다.
li_list=[]
for data1 in data1_list:
    li_list.extend(data1.findAll('img'))

In [218]:
li_list2

['<img src="https://static.ewg.org/skindeep/img/draw_score/score_image1__1_.png" style="border: 0"/>',
 '<img src="https://static.ewg.org/skindeep/img/draw_score/score_image1__1_.png" style="border: 0"/>',
 '<img src="https://static.ewg.org/skindeep/img/draw_score/score_image1__1_.png" style="border: 0"/>',
 '<img src="https://static.ewg.org/skindeep/img/draw_score/score_image1__1_.png" style="border: 0"/>',
 '<img src="https://static.ewg.org/skindeep/img/draw_score/score_image1__1_.png" style="border: 0"/>',
 '<img src="https://static.ewg.org/skindeep/img/draw_score/score_image1__1_.png" style="border: 0"/>',
 '<img src="https://static.ewg.org/skindeep/img/draw_score/score_image1__1_.png" style="border: 0"/>',
 '<img src="https://static.ewg.org/skindeep/img/draw_score/score_image1__1_.png" style="border: 0"/>']

In [217]:
#각 리스트의 원소마다 한개씩 추출하기 위해 해당 위치를 찾아서 추출한다.
lili_list2[1][68:69]

'1'

In [213]:
len(li_list2)

8

In [208]:
li_list2=[]
for x in li_list:
    li_list2.append(str(x))

In [219]:
li_list3=[]
for y in range(8):
    li_list3.append(li_list2[y][68:69])

IndexError: list index out of range

# Selenium 사용하여 ewg 성분 추출하기

## 추출하기 앞서 chrome driver을 깔아야 한다.

In [ ]:
#깔았던 chrome driver 경로를 연결하여 진행
driver=webdriver.Chrome('C:/Users/user/chromedriver')

In [276]:
ewg=pd.DataFrame([],columns=['ingredient','concerns','score'])

In [2]:
driver=webdriver.Chrome('C:/Users/user/chromedriver')
ewg=pd.DataFrame([],columns=['ingredient','concerns','score'])
driver.get('https://www.ewg.org/skindeep/browse/after+sun+product/')

In [273]:
for i in range(2,12):
    driver.implicitly_wait(randint(2,4))
    driver.find_element_by_xpath("""//*//*[@id="table-browse"]/tbody/tr[{0}]/td[3]/a""".format(i)).click()
    a_element=driver.find_elements_by_xpath("//td[@class='firstcol'] |a[@href]")
    ic=[]
    for i in range(len(a_element)):
        ic.append(a_element[i].text)
    ic=[x for x in ic if x]
    span_element=driver.find_elements_by_xpath("//td[@width]  |span[@style]")
    sb=[]
    for i in range(len(span_element)):
        sb.append(span_element[i].text)
    sb=[x for x in sb if x]
    div_element=driver.find_elements_by_xpath("//td[@align]  |div[@*]")
    ssb=[]
    for i in range(len(div_element)):
        ssb.append(div_element[i].text)
    ssb=[x for x in ssb if x]
    
    html=driver.page_source
    soup=BeautifulSoup(html,'html.parser')
    data1_list=soup.findAll('div',{'id':"prod_cntr_score"})
    li_list=[]
    for data1 in data1_list:
        li_list.extend(data1.findAll('img'))
    li_list2=[]
    for x in li_list:
        li_list2.append(str(x))
    li_list3=[]
    for y in range(len(li_list2)):
        li_list3.append(li_list2[y][68:69])
    
    
    ic=pd.Series(ic,name='ingredient')
    sb=pd.Series(sb,name='concerns')
    ssb=pd.Series(ssb,name='score')
    li_list=pd.Series(li_list3,name='score_number')
    first=pd.concat([ic,sb,ssb,li_list],axis=1)
    ewg=ewg.append(first)
    
    
    
    driver.back()

# after suncream

In [4]:
for j in range(0,11,10):
    driver.implicitly_wait(randint(2,4))
    driver.get("""https://www.ewg.org/skindeep/browse.php?category=after_sun_product&&showmore=products&start={0}""".format(j))
    for i in range(2,12):
        driver.find_element_by_xpath("""//*//*[@id="table-browse"]/tbody/tr[{0}]/td[3]/a""".format(i)).click()
        a_element=driver.find_elements_by_xpath("//td[@class='firstcol'] |a[@href]")
        ic=[]
        for i in range(len(a_element)):
            ic.append(a_element[i].text)
        ic=[x for x in ic if x]
        span_element=driver.find_elements_by_xpath("//td[@width]  |span[@style]")
        sb=[]
        for i in range(len(span_element)):
            sb.append(span_element[i].text)
        sb=[x for x in sb if x]
        div_element=driver.find_elements_by_xpath("//td[@align]  |div[@*]")
        ssb=[]
        for i in range(len(div_element)):
            ssb.append(div_element[i].text)
        ssb=[x for x in ssb if x]
        html=driver.page_source
        soup=BeautifulSoup(html,'html.parser')
        data1_list=soup.findAll('div',{'id':"prod_cntr_score"})
        li_list=[]
        for data1 in data1_list:
            li_list.extend(data1.findAll('img'))
        li_list2=[]
        for x in li_list:
            li_list2.append(str(x))
        li_list3=[]
        for y in range(len(li_list2)):
            li_list3.append(li_list2[y][68:69])
        ic=pd.Series(ic,name='ingredient')
        sb=pd.Series(sb,name='concerns')
        ssb=pd.Series(ssb,name='score')
        li_list=pd.Series(li_list3,name='score_number')
        first=pd.concat([ic,sb,ssb,li_list],axis=1)
        ewg=ewg.append(first)
        driver.back()

NameError: name 'ewg' is not defined

In [283]:
ewg.columns=['concerns','ingredient','score','score_number']
ewg1=ewg.copy()

In [284]:
ewg1=ewg1[['ingredient','concerns','score_number','score']]
ewg1

,ingredient,concerns,score_number,score
0,ALOE BARBADENSIS (ALOE VERA) LEAF EXTRACT\nALO...,Use restrictions (meets restrictions and warni...,1,Data: Limited
1,ROSMARINUS OFFICINALIS (ROSEMARY) LEAF WATER\n...,None Identified,1,Data: Limited
2,SODIUM CARBOMER,None Identified,1,Data: None
3,ZANTHOXYLUM PIPERITUM FRUIT EXTRACT,None Identified,1,Data: None
4,USNEA BARBATA (BEARD MOSS)\nUSNEA BARBATA (LIC...,None Identified,1,Data: Limited
5,PULSATILLA KOREANA EXTRACT,None Identified,1,Data: Limited
6,PROPANEDIOL,"Irritation (skin, eyes, or lungs)",1,Data: Fair
7,PHENETHYL ALCOHOL,Use restrictions (meets restrictions and warni...,1,Data: Fair
0,POTASSIUM SORBATE,"Allergies/immunotoxicity, Organ system toxicit...",3,Data: Limited
1,"SILICA, AMORPHOUS",None Identified,1,Data: Good


In [286]:
ewg1.to_csv("After_Sun_Product.csv")

# facial cleanser
맨 끝의 에러는 xpath의 갯수를 10개로 맞춰서 맨끝의 페이지의 제품이 10개가 아니기 때문에 에러가 뜨는 겁니다.

문제가 없는 코드이니 신경쓰지 마세요

In [2]:
driver=webdriver.Chrome('C:/Users/user/chromedriver')
ewg=pd.DataFrame([],columns=['ingredient','concerns','score'])
driver.get('https://www.ewg.org/skindeep/browse/after+sun+product/')

In [11]:
ewg_facial=pd.DataFrame([],columns=['ingredient','concerns','score'])

In [8]:
for j in range(1310,1921,10):
    driver.implicitly_wait(randint(2,4))
    driver.get("""https://www.ewg.org/skindeep/browse.php?category=facial_cleanser&&showmore=products&start={0}""".format(j))
    for i in range(2,12):
        driver.find_element_by_xpath("""//*//*[@id="table-browse"]/tbody/tr[{0}]/td[3]/a""".format(i)).click()
        a_element=driver.find_elements_by_xpath("//td[@class='firstcol'] |a[@href]")
        ic=[]
        for i in range(len(a_element)):
            ic.append(a_element[i].text)
        ic=[x for x in ic if x]
        span_element=driver.find_elements_by_xpath("//td[@width]  |span[@style]")
        sb=[]
        for i in range(len(span_element)):
            sb.append(span_element[i].text)
        sb=[x for x in sb if x]
        div_element=driver.find_elements_by_xpath("//td[@align]  |div[@*]")
        ssb=[]
        for i in range(len(div_element)):
            ssb.append(div_element[i].text)
        ssb=[x for x in ssb if x]
        html=driver.page_source
        soup=BeautifulSoup(html,'html.parser')
        data1_list=soup.findAll('div',{'id':"prod_cntr_score"})
        li_list=[]
        for data1 in data1_list:
            li_list.extend(data1.findAll('img'))
        li_list2=[]
        for x in li_list:
            li_list2.append(str(x))
        li_list3=[]
        for y in range(len(li_list2)):
            li_list3.append(li_list2[y][68:69])
        ic=pd.Series(ic,name='ingredient')
        sb=pd.Series(sb,name='concerns')
        ssb=pd.Series(ssb,name='score')
        li_list=pd.Series(li_list3,name='score_number')
        first=pd.concat([ic,sb,ssb,li_list],axis=1)
        ewg_facial=ewg_facial.append(first)
        driver.back()
        

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*//*[@id="table-browse"]/tbody/tr[7]/td[3]/a"}
  (Session info: chrome=75.0.3770.142)


In [290]:
ewg_facial.to_csv("Facial_Cleanser.csv")

In [9]:
ewg_facial

,concerns,ingredient,score,score_number
0,"Miscellaneous, Allergies/immunotoxicity, Irrit...",FRAGRANCE,Data: Fair,8
1,"Enhanced skin absorption, Organ system toxicit...",PEG-7 GLYCERYL COCOATE,Data: None,4
2,"Use restrictions, Allergies/immunotoxicity, Ec...",COCAMIDOPROPYL BETAINE,Data: Good,4
3,"Irritation (skin, eyes, or lungs), Occupationa...",PHENOXYETHANOL,Data: Limited,4
4,Use restrictions,GLYCERIN,Data: Good,2
5,"Use restrictions, Contamination concerns (PEST...",ORYZA SATIVA (RICE) STARCH,Data: Fair,2
6,None Identified,DISODIUM LAUROAMPHODIACETATE,Data: None,1
7,None Identified,SODIUM METHYL COCOYL TAURATE,Data: None,1
8,None Identified,PHENYLPROPANOL,Data: Limited,1
9,Ecotoxicology,GLYCOL STEARATE,Data: Limited,1


In [10]:
ewg_facial.to_csv("Facial_Cleanser3.csv")

# Toners/astringents

In [20]:
ewg_toners=pd.DataFrame([],columns=['ingredient','concerns','score'])

In [16]:
for j in range(0,741,10):
    driver.implicitly_wait(randint(2,4))
    driver.get("""https://www.ewg.org/skindeep/browse.php?category=toners_astringents&&showmore=products&start={0}""".format(j))
    for i in range(2,12):
        driver.find_element_by_xpath("""//*//*[@id="table-browse"]/tbody/tr[{0}]/td[3]/a""".format(i)).click()
        a_element=driver.find_elements_by_xpath("//td[@class='firstcol'] |a[@href]")
        ic=[]
        for i in range(len(a_element)):
            ic.append(a_element[i].text)
        ic=[x for x in ic if x]
        span_element=driver.find_elements_by_xpath("//td[@width]  |span[@style]")
        sb=[]
        for i in range(len(span_element)):
            sb.append(span_element[i].text)
        sb=[x for x in sb if x]
        div_element=driver.find_elements_by_xpath("//td[@align]  |div[@*]")
        ssb=[]
        for i in range(len(div_element)):
            ssb.append(div_element[i].text)
        ssb=[x for x in ssb if x]
        html=driver.page_source
        soup=BeautifulSoup(html,'html.parser')
        data1_list=soup.findAll('div',{'id':"prod_cntr_score"})
        li_list=[]
        for data1 in data1_list:
            li_list.extend(data1.findAll('img'))
        li_list2=[]
        for x in li_list:
            li_list2.append(str(x))
        li_list3=[]
        for y in range(len(li_list2)):
            li_list3.append(li_list2[y][68:69])
        ic=pd.Series(ic,name='ingredient')
        sb=pd.Series(sb,name='concerns')
        ssb=pd.Series(ssb,name='score')
        li_list=pd.Series(li_list3,name='score_number')
        first=pd.concat([ic,sb,ssb,li_list],axis=1)
        ewg_toners=ewg_toners.append(first)
        driver.back()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*//*[@id="table-browse"]/tbody/tr[11]/td[3]/a"}
  (Session info: chrome=75.0.3770.142)


In [18]:
ewg_toners.to_csv('ewg_toners.csv')

# eye cream

In [19]:
ewg_eye=pd.DataFrame([],columns=['ingredient','concerns','score'])

In [21]:
for j in range(0,761,10):
    driver.implicitly_wait(randint(2,4))
    driver.get("""https://www.ewg.org/skindeep/browse.php?category=around-eye_cream&&showmore=products&start={0}""".format(j))
    for i in range(2,12):
        driver.find_element_by_xpath("""//*//*[@id="table-browse"]/tbody/tr[{0}]/td[3]/a""".format(i)).click()
        a_element=driver.find_elements_by_xpath("//td[@class='firstcol'] |a[@href]")
        ic=[]
        for i in range(len(a_element)):
            ic.append(a_element[i].text)
        ic=[x for x in ic if x]
        span_element=driver.find_elements_by_xpath("//td[@width]  |span[@style]")
        sb=[]
        for i in range(len(span_element)):
            sb.append(span_element[i].text)
        sb=[x for x in sb if x]
        div_element=driver.find_elements_by_xpath("//td[@align]  |div[@*]")
        ssb=[]
        for i in range(len(div_element)):
            ssb.append(div_element[i].text)
        ssb=[x for x in ssb if x]
        html=driver.page_source
        soup=BeautifulSoup(html,'html.parser')
        data1_list=soup.findAll('div',{'id':"prod_cntr_score"})
        li_list=[]
        for data1 in data1_list:
            li_list.extend(data1.findAll('img'))
        li_list2=[]
        for x in li_list:
            li_list2.append(str(x))
        li_list3=[]
        for y in range(len(li_list2)):
            li_list3.append(li_list2[y][68:69])
        ic=pd.Series(ic,name='ingredient')
        sb=pd.Series(sb,name='concerns')
        ssb=pd.Series(ssb,name='score')
        li_list=pd.Series(li_list3,name='score_number')
        first=pd.concat([ic,sb,ssb,li_list],axis=1)
        ewg_eye=ewg_eye.append(first)
        driver.back()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*//*[@id="table-browse"]/tbody/tr[3]/td[3]/a"}
  (Session info: chrome=75.0.3770.142)


In [23]:
ewg_eye.to_csv('ewg_eye.csv')

# Men facial

In [12]:
ewg_men_facial=pd.DataFrame([],columns=['ingredient','concerns','score'])

In [13]:
for j in range(1692,3411,10):
    driver.implicitly_wait(randint(2,4))
    driver.get("""https://www.ewg.org/skindeep/browse.php?category=facial_moisturizer%3B%3Btreatment&&showmore=products&start={0}""".format(j))
    for i in range(2,12):
        driver.find_element_by_xpath("""//*//*[@id="table-browse"]/tbody/tr[{0}]/td[3]/a""".format(i)).click()
        a_element=driver.find_elements_by_xpath("//td[@class='firstcol'] |a[@href]")
        ic=[]
        for i in range(len(a_element)):
            ic.append(a_element[i].text)
        ic=[x for x in ic if x]
        span_element=driver.find_elements_by_xpath("//td[@width]  |span[@style]")
        sb=[]
        for i in range(len(span_element)):
            sb.append(span_element[i].text)
        sb=[x for x in sb if x]
        div_element=driver.find_elements_by_xpath("//td[@align]  |div[@*]")
        ssb=[]
        for i in range(len(div_element)):
            ssb.append(div_element[i].text)
        ssb=[x for x in ssb if x]
        html=driver.page_source
        soup=BeautifulSoup(html,'html.parser')
        data1_list=soup.findAll('div',{'id':"prod_cntr_score"})
        li_list=[]
        for data1 in data1_list:
            li_list.extend(data1.findAll('img'))
        li_list2=[]
        for x in li_list:
            li_list2.append(str(x))
        li_list3=[]
        for y in range(len(li_list2)):
            li_list3.append(li_list2[y][68:69])
        ic=pd.Series(ic,name='ingredient')
        sb=pd.Series(sb,name='concerns')
        ssb=pd.Series(ssb,name='score')
        li_list=pd.Series(li_list3,name='score_number')
        first=pd.concat([ic,sb,ssb,li_list],axis=1)
        ewg_men_facial=ewg_men_facial.append(first)
        driver.back()

In [14]:
ewg_men_facial.to_csv('ewg_men_facial2.csv')

# men facial cleanser

In [3]:
ewg_men_facial_cleanser=pd.DataFrame([],columns=['ingredient','concerns','score'])

In [4]:
for j in range(0,1921,10):
    driver.implicitly_wait(randint(2,4))
    driver.get("""https://www.ewg.org/skindeep/browse.php?category=facial_cleanser&&showmore=products&start={0}""".format(j))
    for i in range(2,12):
        driver.find_element_by_xpath("""//*//*[@id="table-browse"]/tbody/tr[{0}]/td[3]/a""".format(i)).click()
        a_element=driver.find_elements_by_xpath("//td[@class='firstcol'] |a[@href]")
        ic=[]
        for i in range(len(a_element)):
            ic.append(a_element[i].text)
        ic=[x for x in ic if x]
        span_element=driver.find_elements_by_xpath("//td[@width]  |span[@style]")
        sb=[]
        for i in range(len(span_element)):
            sb.append(span_element[i].text)
        sb=[x for x in sb if x]
        div_element=driver.find_elements_by_xpath("//td[@align]  |div[@*]")
        ssb=[]
        for i in range(len(div_element)):
            ssb.append(div_element[i].text)
        ssb=[x for x in ssb if x]
        html=driver.page_source
        soup=BeautifulSoup(html,'html.parser')
        data1_list=soup.findAll('div',{'id':"prod_cntr_score"})
        li_list=[]
        for data1 in data1_list:
            li_list.extend(data1.findAll('img'))
        li_list2=[]
        for x in li_list:
            li_list2.append(str(x))
        li_list3=[]
        for y in range(len(li_list2)):
            li_list3.append(li_list2[y][68:69])
        ic=pd.Series(ic,name='ingredient')
        sb=pd.Series(sb,name='concerns')
        ssb=pd.Series(ssb,name='score')
        li_list=pd.Series(li_list3,name='score_number')
        first=pd.concat([ic,sb,ssb,li_list],axis=1)
        ewg_men_facial_cleanser=ewg_men_facial_cleanser.append(first)
        driver.back()

C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*//*[@id="table-browse"]/tbody/tr[8]/td[3]/a"}
  (Session info: chrome=75.0.3770.142)


In [5]:
ewg_men_facial_cleanser.to_csv('ewg_men_facial_cleanser.csv')

# facial

In [8]:
ewg_facial_treat=pd.DataFrame([],columns=['ingredient','concerns','score'])

In [9]:
for j in range(1720,3401,10):
    driver.implicitly_wait(randint(2,4))
    driver.get("""https://www.ewg.org/skindeep/browse.php?category=facial_moisturizer%3B%3Btreatment&&showmore=products&start={0}""".format(j))
    for i in range(2,12):
        driver.find_element_by_xpath("""//*//*[@id="table-browse"]/tbody/tr[{0}]/td[3]/a""".format(i)).click()
        a_element=driver.find_elements_by_xpath("//td[@class='firstcol'] |a[@href]")
        ic=[]
        for i in range(len(a_element)):
            ic.append(a_element[i].text)
        ic=[x for x in ic if x]
        span_element=driver.find_elements_by_xpath("//td[@width]  |span[@style]")
        sb=[]
        for i in range(len(span_element)):
            sb.append(span_element[i].text)
        sb=[x for x in sb if x]
        div_element=driver.find_elements_by_xpath("//td[@align]  |div[@*]")
        ssb=[]
        for i in range(len(div_element)):
            ssb.append(div_element[i].text)
        ssb=[x for x in ssb if x]
        html=driver.page_source
        soup=BeautifulSoup(html,'html.parser')
        data1_list=soup.findAll('div',{'id':"prod_cntr_score"})
        li_list=[]
        for data1 in data1_list:
            li_list.extend(data1.findAll('img'))
        li_list2=[]
        for x in li_list:
            li_list2.append(str(x))
        li_list3=[]
        for y in range(len(li_list2)):
            li_list3.append(li_list2[y][68:69])
        ic=pd.Series(ic,name='ingredient')
        sb=pd.Series(sb,name='concerns')
        ssb=pd.Series(ssb,name='score')
        li_list=pd.Series(li_list3,name='score_number')
        first=pd.concat([ic,sb,ssb,li_list],axis=1)
        ewg_facial_treat=ewg_facial_treat.append(first)
        driver.back()
        


In [10]:
ewg_facial_treat.to_csv('ewg_facial_treat3.csv')

In [8]:
ewg_moisture=pd.DataFrame([],columns=['ingredient','concerns','score'])

In [9]:
for j in range(3010,3801,10):
    driver.implicitly_wait(randint(2,4))
    driver.get("""https://www.ewg.org/skindeep/browse.php?category=moisturizer&&showmore=products&start={0}""".format(j))
    for i in range(2,12):
        driver.find_element_by_xpath("""//*//*[@id="table-browse"]/tbody/tr[{0}]/td[3]/a""".format(i)).click()
        a_element=driver.find_elements_by_xpath("//td[@class='firstcol'] |a[@href]")
        ic=[]
        for i in range(len(a_element)):
            ic.append(a_element[i].text)
        ic=[x for x in ic if x]
        span_element=driver.find_elements_by_xpath("//td[@width]  |span[@style]")
        sb=[]
        for i in range(len(span_element)):
            sb.append(span_element[i].text)
        sb=[x for x in sb if x]
        div_element=driver.find_elements_by_xpath("//td[@align]  |div[@*]")
        ssb=[]
        for i in range(len(div_element)):
            ssb.append(div_element[i].text)
        ssb=[x for x in ssb if x]
        html=driver.page_source
        soup=BeautifulSoup(html,'html.parser')
        data1_list=soup.findAll('div',{'id':"prod_cntr_score"})
        li_list=[]
        for data1 in data1_list:
            li_list.extend(data1.findAll('img'))
        li_list2=[]
        for x in li_list:
            li_list2.append(str(x))
        li_list3=[]
        for y in range(len(li_list2)):
            li_list3.append(li_list2[y][68:69])
        ic=pd.Series(ic,name='ingredient')
        sb=pd.Series(sb,name='concerns')
        ssb=pd.Series(ssb,name='score')
        li_list=pd.Series(li_list3,name='score_number')
        first=pd.concat([ic,sb,ssb,li_list],axis=1)
        ewg_moisture=ewg_moisture.append(first)
        driver.back()
    #179부터 다시 하면 됨

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*//*[@id="table-browse"]/tbody/tr[8]/td[3]/a"}
  (Session info: chrome=75.0.3770.142)


In [10]:
ewg_moisture.to_csv('ewg_moisture3.csv')

In [14]:
driver.close()